In [1]:

import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
filename = 'data/Wonderland.txt'
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [3]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [4]:
n_chars = len(raw_text)
n_vocab = len(chars)
print( "Total Characters: ", n_chars)
print( "Total Vocab: ", n_vocab)

Total Characters:  163815
Total Vocab:  60


In [5]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  163715


In [6]:
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode
y = np_utils.to_categorical(dataY)

In [7]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [8]:
# define the checkpoint
filepath="data/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [13]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
163715/163715 [==============================] - 1048s 6ms/step - loss: 2.8178
Epoch 2/20
163715/163715 [==============================] - 1069s 7ms/step - loss: 2.7197
Epoch 3/20
163715/163715 [==============================] - 1066s 7ms/step - loss: 2.6482
Epoch 4/20
163715/163715 [==============================] - 1075s 7ms/step - loss: 2.5822
Epoch 5/20
163715/163715 [==============================] - 1062s 6ms/step - loss: 2.5211
Epoch 6/20
163715/163715 [==============================] - 1060s 6ms/step - loss: 2.4672
Epoch 7/20
163715/163715 [==============================] - 1079s 7ms/step - loss: 2.4207
Epoch 8/20
163715/163715 [==============================] - 1067s 7ms/step - loss: 2.3719
Epoch 9/20
163715/163715 [==============================] - 1072s 7ms/step - loss: 2.3297
Epoch 10/20
163715/163715 [==============================] - 1052s 6ms/step - loss: 2.2901
Epoch 11/20
163715/163715 [==============================] - 1051s 6ms/step - loss: 2.2515
Epoch 12

In [10]:
filename = "data/weights-improvement-19-1.9435.hdf5"
model.save_weights(filename)
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [20]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print( "Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print( "\nDone.")

Seed:
"  from her as hard as it could go, and making quite a commotion in
the pool as it went.

so she calle "
d to ter whate whe world oadt to the woile oa then and toen a getd wan a lootne oo the tait, and the war not in the bane and the sabbit sat sh the tabte oi the gad nerer here the was so toe thee  the was not in the bane and the sabbit sat sh the tabte oi the garter was no the tine the was soenk again, and the whrt here toiee the whrt wall to the woide the was so tone the was so toye 
the had nov oo the taate of the war so toenk ano hor hoa to the tooe of the garter, and was solniig an inr an she wooke. and tae toei it har hnat the white rabbit was soe oant of the whrt of the garter, and whs wored toen it tas an anl oo the toie. ‘ho wou den to you do wou hane to tea that iave a can! io wou den’ wouhd the want on an inrssns,’

‘h woode than to ae a cetd wail to,’ said alice, ‘aod the mors oi the garter wouh a cian winh i sai soene the wante  and the world bale wait in she bane a

In [21]:
#influenced by https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/x